In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab


1、Download the training data

In [ ]:
!wget https://www.dropbox.com/s/t1a61exycm537v5/clips-data-2020.zip?dl=0
!mv clips-data-2020.zip?dl=0 clips-data-2020.zip
!unzip -o clips-data-2020.zip > /dev/null
print("complete")

--2020-04-14 04:46:04--  https://www.dropbox.com/s/t1a61exycm537v5/clips-data-2020.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/t1a61exycm537v5/clips-data-2020.zip [following]
--2020-04-14 04:46:04--  https://www.dropbox.com/s/raw/t1a61exycm537v5/clips-data-2020.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc719f27e8750e87efdf43266dca.dl.dropboxusercontent.com/cd/0/inline/A1w2nw-N3ZLYbJmt4Bb6xuaiGZLp-7u1k65tXVBL7nXcfPo5t7Q27ZjgDF9-W5yA62GtS3kTxSmC1id7ghPFTM7nH72W2uo5UJAo5yFBj23gpvs0OpA209IG7ODGHo0GkFo/file# [following]
--2020-04-14 04:46:04--  https://uc719f27e8750e87efdf43266dca.dl.dropboxusercontent.com/cd/0/inline/A1w2nw-N3ZLYbJmt4Bb6xuaiGZLp-7u1k65tXVBL7nXcfPo5t7Q27ZjgDF9-W5yA62GtS3kTxSmC1id7ghPFTM7n

2、Next, we build a dataframe that contains the filenames and the clip_count.  


In [ ]:
import pandas as pd

df = pd.read_csv(
    "/content/drive/My Drive/Colab Notebooks/kaggle_paperclip/train.csv", 
    na_values=['NA', '?'])

df['filename']="clips-"+df["id"].astype(str)+".png"

3、Separate into a training and validation (for early stopping)

In [ ]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df) * TRAIN_PCT)

df_train = df[0:TRAIN_CUT]
df_validate = df[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 18000
Validate size: 2000


4、Create DataGenerators for training and validation. 

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

IMAGES_DIR = "/content/clips"

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(150, 150),
        batch_size=16,
        class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(150, 150),
        class_mode='other')

Found 18000 validated image filenames.
Found 2000 validated image filenames.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.summary()
epoch_steps = 1000 # len(df_train)
validation_steps = len(df_validate)
model.compile(loss = 'mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  steps_per_epoch=epoch_steps, validation_steps=validation_steps, verbose = 1, 
  callbacks=[monitor], validation_data=val_generator, epochs=25)
# 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       2

5、Generate Submission File

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),

    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),

    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.summary()
epoch_steps = 1000 # len(df_train)
validation_steps = len(df_validate)
model.compile(loss = 'mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
history = model.fit(train_generator,  
  steps_per_epoch=epoch_steps, validation_steps=validation_steps, verbose = 1, 
  callbacks=[monitor], validation_data=val_generator, epochs=25)
#

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 52, 52, 256)       295168    
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 50, 50, 256)       590080    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 25, 25, 256)      

In [ ]:
df_test = pd.read_csv(
    "/content/drive/My Drive/Colab Notebooks/kaggle_paperclip/test.csv", 
    na_values=['NA', '?'])

df_test['filename']="clips-"+df_test["id"].astype(str)+".png"

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR,
        x_col="filename",
        batch_size=1,
        shuffle=False,
        target_size=(150, 150),
        class_mode=None)

Found 5000 validated image filenames.


In [ ]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

In [ ]:
df_submit = pd.DataFrame({'id':df_test['id'],'clip_count':pred.flatten()})

In [ ]:
df_submit.to_csv("/content/drive/My Drive/Colab Notebooks/kaggle_paperclip/submit5.csv",index=False)